In [1]:
import os
import re
import torch
import json
import nltk
import unicodedata
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datasets import Dataset
from typing import List, Tuple
from transformers import AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

Cuda and stuff

In [ ]:
# Check whether CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.rand(3, 3).to(device)
print(f'Tensor is on: {x.device}')

In [ ]:
# print cuda version
print(torch.version.cuda)

In [ ]:
# check the current working directory
#os.chdir('C:/Users/lamem/OneDrive/Documents/GHD/Microsoft-Learn-Location-Mention-Recognition-Challenge')
os.chdir('C:/Users/abelm/OneDrive/Documents/GitHub/Microsoft-Learn-Location-Mention-Recognition-Challenge')
print(os.getcwd())

In [ ]:
new = pd.read_csv("lewa/Train_1.csv")
new.head()

In [6]:
# initialize stopwords
stop_words = set(stopwords.words('english'))

# text clean helper function
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # remove mentions only
    text = re.sub(r'@\w+', '', text)
    
    # Remove special characters, numbers, and punctuations, keeping spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove stopwords
    words = text.split()  # Split by spaces without tokenizing
    cleaned_words = [word for word in words if word not in stop_words]
    
    # Join words back into a single string
    return ' '.join(cleaned_words)

In [ ]:
# Load your  
df_t = new.copy() 
df_t = df_t.dropna(subset = "text")
# clean the text
df_t['text'] = df_t['text'].apply(clean_text)
df_t.head()

In [ ]:
df_t.shape

In [9]:
# separate data into train and dev
trainData, devData = train_test_split(df_t, test_size=0.15, random_state=42)

# save data to a csv tainData
trainData.to_csv('lewa/Train-dropna.csv', index=False)

# save data to a csv devData
devData.to_csv('lewa/Dev-dropna.csv', index=False)

In [10]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Open the CSV file and create a list of dictionaries
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = [row for row in csv_reader]

    # Write the list of dictionaries to a JSON file
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage
#csv_file_path = 'testData/Train-dropna.csv'  # Path to your CSV file
#json_file_path = 'testData/output_file.json'  # Path where the JSON will be saved
csv_to_json('lewa/Train-dropna.csv', 'lewa/train_file.json')
csv_to_json('lewa/Dev-dropna.csv', 'lewa/dev_file.json')


IDENTIFY location from sentences 

In [11]:
import spacy
import string

# Load spaCy's English model
nlp = spacy.load("en_core_web_lg")

def remove_hashtag(text):
    # Find hashtags and process each one
    return re.sub(r'#(\w+)', lambda m: ' '.join(re.findall(r'[A-Z][^A-Z]*|[a-z]+', m.group(1))), text)

# Function to clean and preprocess text (tokenization, normalization, lemmatization)
def preprocess_text(text):
    # Remove special characters (keeping alphanumeric characters and spaces)
    text = remove_hashtag(text)  # Remove hashtags and separate words
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    doc = nlp(text)
    cleaned_tokens = []
    for token in doc:
        # Normalize: Convert to lowercase and remove punctuation
        if token.text not in string.punctuation:
            cleaned_tokens.append(token.lemma_.lower())  # Lemmatize and lowercase
    return " ".join(cleaned_tokens)

# Function to extract locations as entities after preprocessing
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ == 'GPE':  # 'GPE' is for locations
            entities.append({
                'entity': ent.text,
                'label': ent.label_,
                'start_char': ent.start_char,
                'end_char': ent.end_char
            })
    return entities

# Function to process a JSON file
def process_json_file(file_path):
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.DataFrame(data)

    # Preprocess text and extract entities
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    df['entities'] = df['cleaned_text'].apply(extract_entities)

    return df

# Paths to your two JSON files
file_path_1 = 'lewa/train_file.json'
file_path_2 = 'lewa/dev_file.json'

# Process both files
df1 = process_json_file(file_path_1)
df2 = process_json_file(file_path_2)


# If you want to save them separately, you can use the individual DataFrames:
# Save the updated data for file1
with open('lewa/train_entities.json', 'w') as f:
    json.dump(df1.to_dict(orient='records'), f, indent=4)

# Save the updated data for file2
with open('lewa/dev_entities.json', 'w') as f:
    json.dump(df2.to_dict(orient='records'), f, indent=4)

## MERGE THE EXTRACTED LOCATIONS WITH EXISTING LOCATION COLUMN

In [12]:
# helper function to merge location and entities, taking file paths for input and output
def merge_entities(input_file_path, output_file_path):
    # Read the unmerged JSON file
    with open(input_file_path, 'r') as infile:
        data = json.load(infile)
    
    for entry in data:
        location = entry.get("location")
        entities = entry.get("entities", [])
        
        # Create the merged_entities field with only entities present in the entities list
        entry["merged_entities"] = [ent["entity"] for ent in entities]

        merged_entities = [ent["entity"] for ent in entities]

        # If location is present and valid, merge it with entities
        if location:
            merged_entities.append(location)

        # Update the entry with merged entities
        entry["merged_entities"] = merged_entities
    
    # Write the updated data to the output JSON file
    with open(output_file_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)


In [13]:
# Paths to your two JSON files
input_file_path = 'lewa/train_entities.json'
output_file_path = 'lewa/train_merged_entities.json'

# Merge entities for the first file
merge_entities(input_file_path, output_file_path)

# Paths to your two JSON files
input_file_path = 'lewa/dev_entities.json'
output_file_path = 'lewa/dev_merged_entities.json'

# Merge entities for the first file
merge_entities(input_file_path, output_file_path)

BIOES TAGGING

In [14]:
# Helper function to read JSON file and return a DataFrame with specific columns
def json_to_csv(input_file_path, output_csv_path):
    # Read the JSON file
    with open(input_file_path, 'r') as infile:
        data = json.load(infile)
    
    # Create a list to hold the data for the DataFrame
    rows = []
    
    for entry in data:
        tweet_id = entry.get("tweet_id")
        cleaned_text = entry.get("cleaned_text")
        merged_entities = entry.get("merged_entities", [])
        
        # Convert the list of entities into a single string (join them with a comma and space)
        location = ' '.join(merged_entities) if merged_entities else None
        
        # Add the row to the list
        rows.append({
            "tweet_id": tweet_id,
            "text": cleaned_text,  # Rename cleaned_text to text
            "location": location  # Convert merged_entities to a plain text string
        })
    
    # Create a DataFrame from the list of rows
    df = pd.DataFrame(rows)
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

In [15]:
# save merged_train_entities to a csv
train_file_path = 'lewa/train_merged_entities.json'
train_csv_path = 'lewa/train_merged_entities.csv'
json_to_csv(train_file_path, train_csv_path)

# save merged_dev_entities to a csv
dev_file_path = 'lewa/dev_merged_entities.json' 
dev_csv_path = 'lewa/dev_merged_entities.csv'
json_to_csv(dev_file_path, dev_csv_path)

tagging

In [21]:
def bioes_tagging(text, merged_entities):
    """
    Apply BIOES tagging to a tokenized text based on merged entity spans.
    
    :param text: List of tokenized words.
    :param merged_entities: List of merged location entities in the sentence (already a list).
    :return: List of tuples (word, tag) in BIOES format.
    """
    tags = ['O'] * len(text)
    
    # Process each merged entity (location)
    for entity in merged_entities:
        entity_tokens = entity.split()  # Assuming each entity is a string of tokens
        entity_length = len(entity_tokens)

        # Apply BIOES tags for each token in the entity
        for i, token in enumerate(entity_tokens):
            try:
                token_index = text.index(token)
            except ValueError:
                continue  # Skip tokens not found in the sentence

            if entity_length == 1:
                tags[token_index] = 'S-LOC'
            elif i == 0:
                tags[token_index] = 'B-LOC'
            elif i == entity_length - 1:
                tags[token_index] = 'E-LOC'
            else:
                tags[token_index] = 'I-LOC'
    
    return list(zip(text, tags))

def process_json(input_file, output_file):
    # Read the JSON file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    output_data = []

    # Process each entry in the JSON file
    for entry in data:
        text = entry['text']  # Assuming 'text' field has the sentence
        merged_entities = entry.get('merged_entities', [])  # No split needed, it's already a list
        
        # Tokenize the text (basic tokenization)
        tokens = text.split()

        # Apply BIOES tagging
        tagged_tokens = bioes_tagging(tokens, merged_entities)
        
        # Prepare output format
        for token, tag in tagged_tokens:
            output_data.append(f"{token}\t{tag}")
        output_data.append("")  # New line after each sentence

    # Write the output to a text file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(output_data))

In [22]:
# Example usage
process_json('lewa/train_merged_entities.json', 'lewa/train_bioes.txt')
process_json('lewa/dev_merged_entities.json', 'lewa/dev_bioes.txt')

## READ BIEOS

In [23]:
def read_bioes_file(file_path):
    sentences, labels = [], []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            sentence, label = [], []
            for line in file:
                if line.strip():
                    try:
                        word, tag = line.strip().split()
                        sentence.append(word)
                        label.append(tag)
                    except ValueError:
                        print(f"Skipping malformed line: {line.strip()}")
                else:
                    if sentence and label:
                        sentences.append(sentence)
                        labels.append(label)
                    sentence, label = [], []
        return sentences, labels
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except UnicodeDecodeError as e:
        print(f"Unicode decoding error: {e}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        
# read bieos files
train_sentences, train_labels = read_bioes_file('lewa/train_bioes.txt')
dev_sentences, dev_labels = read_bioes_file('lewa/dev_bioes.txt')

In [ ]:
print("Train sentences: ", len(train_sentences))
print("Train labels: ", len(train_labels))
print("Dev sentences: ", len(dev_sentences))
print("Dev labels: ", len(dev_labels))
print()

print("Train sentences: \n", train_sentences[0])
print("Train labels: \n", train_labels[0])
print("Dev sentences: \n", dev_sentences[0])
print("Dev labels: \n", dev_labels[0])

## SET UP THE LABEL MAPPING

In [ ]:
# Set up label mapping
all_labels = set()

for labels in train_labels + dev_labels:
    all_labels.update(labels)

label_list = sorted(list(all_labels))

label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

print(label2id)
print(id2label)

In [ ]:
# Convert train_labels to IDs
train_labels_ids = [[label2id[label] for label in sentence_labels] for sentence_labels in train_labels]
dev_labels_ids = [[label2id[label] for label in sentence_labels] for sentence_labels in dev_labels]

# Example usage
print("Original first sentence labels:", train_labels[0])
print("Converted first sentence label IDs:", train_labels_ids[0])

print("Valid length: ", len(train_labels) == len(train_labels_ids))

## PREPROCESS SENTENCE AND LABEL FROM BIOES TO INDEX

In [29]:
def preprocess_sentence_and_labels(sentence, labels):
    processed_sentence = []
    processed_labels = []

    for word, label in zip(sentence, labels):
        # Remove words with special characters or numbers
        if not re.match(r'^[a-zA-Z]+$', word):
            continue

        # If the word is not empty after processing, keep it and its label
        if word:
            processed_sentence.append(word)
            processed_labels.append(label)

    return processed_sentence, processed_labels

# Process the training data
processed_train_sentences = []
processed_train_labels_ids = []

# Process the training data
processed_dev_sentences = []
processed_dev_labels_ids = []

for sentence, labels in zip(train_sentences, train_labels_ids):
    proc_sentence, proc_labels = preprocess_sentence_and_labels(sentence, labels)
    processed_train_sentences.append(proc_sentence)
    processed_train_labels_ids.append(proc_labels)

for sentence, labels in zip(dev_sentences, dev_labels_ids):
    proc_sentence, proc_labels = preprocess_sentence_and_labels(sentence, labels)
    processed_dev_sentences.append(proc_sentence)
    processed_dev_labels_ids.append(proc_labels)

In [ ]:
# Print an example to compare
print("Original sentence:", train_sentences[2])
print("Original labels:", train_labels_ids[2])
print("\nProcessed sentence:", processed_train_sentences[2])
print(len(processed_train_sentences[2]))
print("Processed labels:", processed_train_labels_ids[2])
print(len(processed_train_labels_ids[2]))

# Print some statistics
original_word_count = sum(len(sentence) for sentence in train_sentences)
processed_word_count = sum(len(sentence) for sentence in processed_train_sentences)
print(f"\nOriginal word count: {original_word_count}")
print(f"Processed word count: {processed_word_count}")
print(f"Removed {original_word_count - processed_word_count} words")

In [ ]:
# Load the tokenizer
CT_M3_Complete_tokenizer = AutoTokenizer.from_pretrained("crisistransformers/CT-M3-Complete")

def tokenize_and_adjust_labels(sentence: List[str], labels: List[int], tokenizer, max_length: int) -> Tuple[List[int], List[int]]:
    tokenized_input = tokenizer(
        sentence,
        is_split_into_words=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # Debugging: Print the shape of the tokenized input
    print("Tokenized input shape:", tokenized_input["input_ids"].shape)

    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0].tolist())  # Convert to list

    # Debugging: Check tokens and their count
    print("Tokens:", tokens)
    print("Number of tokens:", len(tokens))

    updated_labels = []
    current_label_idx = 0

    for token in tokens:
        if token in ['<s>', '</s>', '<unk>']:
            updated_labels.append(-100)
        elif token.endswith('@@'):  # Handle word pieces ending with @@
            if current_label_idx < len(labels):
                updated_labels.append(labels[current_label_idx])
        else:
            if current_label_idx < len(labels):
                updated_labels.append(labels[current_label_idx])
                current_label_idx += 1
            else:
                updated_labels.append(-100)  # Handle case when there are no more labels

    return tokenized_input["input_ids"][0].tolist(), updated_labels  # Return as list

# Set the maximum length for tokenization
max_length = 130  # Adjust this as needed

# Apply the function to all sentences and labels
tokenized_train_inputs = []
adjusted_train_labels = []

tokenized_dev_inputs = []
adjusted_dev_labels = []

for sentence, labels in zip(processed_train_sentences, processed_train_labels_ids):
    input_ids, adjusted_labels = tokenize_and_adjust_labels(sentence, labels, CT_M3_Complete_tokenizer, max_length)
    tokenized_train_inputs.append(input_ids)
    adjusted_train_labels.append(adjusted_labels)

for sentence, labels in zip(processed_dev_sentences, processed_dev_labels_ids):
    input_ids, adjusted_labels = tokenize_and_adjust_labels(sentence, labels, CT_M3_Complete_tokenizer, max_length)
    tokenized_dev_inputs.append(input_ids)
    adjusted_dev_labels.append(adjusted_labels)

In [ ]:
# Print an example to verify
print("Original sentence:", processed_train_sentences[2])
print("Original labels:", processed_train_labels_ids[2])
print("\nTokenized input:", tokenized_train_inputs[2])
print("Adjusted labels:", adjusted_train_labels[2])

# Verify lengths
print("\nLength of tokenized input:", len(tokenized_train_inputs[2]))
print("Length of adjusted labels:", len(adjusted_train_labels[2]))

# Print some statistics
original_sentence_count = len(input_ids)
tokenized_sentence_count = len(tokenized_train_inputs)
print(f"\nNumber of original sentences: {original_sentence_count}")
print(f"Number of tokenized sentences: {tokenized_sentence_count}")

average_original_length = sum(len(s) for s in processed_train_sentences) / original_sentence_count
average_tokenized_length = sum(len(s) for s in tokenized_train_inputs) / tokenized_sentence_count
print(f"\nAverage original sentence length: {average_original_length:.2f}")
print(f"Average tokenized sentence length: {average_tokenized_length:.2f}")

In [ ]:
print(len(input_ids), len(adjusted_labels))  # Check lengths here

## LOAD DATASET

In [34]:
# Convert to datasets
tokenized_train = Dataset.from_dict({
    "input_ids": tokenized_train_inputs,
    "labels": adjusted_train_labels
})
tokenized_dev = Dataset.from_dict({
    "input_ids": tokenized_dev_inputs,
    "labels": adjusted_dev_labels
})

# Set up label mapping
all_labels = set()

for labels in train_labels + dev_labels:
    all_labels.update(labels)

label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

labels = sorted(list(all_labels))

label_list = sorted(list(all_labels))

# MODEL

## MODEL CONFIGURATION

In [ ]:
model_name = "crisistransformers/CT-M3-Complete"

# Update model configuration
config = AutoConfig.from_pretrained(model_name)
config.num_labels = len(label_list)
config.id2label = id2label
config.label2id = label2id

CT_M3_Complete_model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)
CT_M3_Complete_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [36]:
def compute_metrics(p):
    predictions, labels = p
    print(f"Predictions shape: {predictions.shape}, Labels shape: {labels.shape}")
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = precision_recall_fscore_support(sum(true_labels, []), sum(true_predictions, []), average='weighted')
    return {
        "precision": results[0],
        "recall": results[1],
        "f1": results[2],
    }

## TRAINING

In [37]:
# Set up model and tokenizer
model_name = "crisistransformers/CT-M3-Complete"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = len(label_list)
config.id2label = id2label
config.label2id = label2id

In [ ]:
# Set up data collator
data_collator = DataCollatorForTokenClassification(tokenizer=CT_M3_Complete_tokenizer, padding=True)

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        # Print input shapes here
        print(f"Inputs: {inputs['input_ids'].shape}, Labels: {inputs['labels'].shape}")
        return super().training_step(model, inputs)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="lewa/model/CrisisTransformers",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Accumulate gradients
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch",  # Use PyTorch's AdamW implementation
    logging_steps=100,  # Reduce logging frequency
    save_total_limit=2,  # Keep only the last 2 checkpoints
    report_to='none',  # Disable logging to wandb
)

# Set up trainer
CustomTrainer = Trainer(
    model=CT_M3_Complete_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=CT_M3_Complete_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
print(f"Training dataset size: {len(tokenized_train_inputs)}")
print(f"First training input shape: {len(tokenized_train_inputs[0])}")
print(f"First training label shape: {len(adjusted_train_labels[0])}")

In [ ]:
# Start training
CustomTrainer.train()

## EVALUATE THE MODEL

In [ ]:
# Evaluate the model
eval_results = CustomTrainer.evaluate()
print(eval_results)

In [ ]:
# After training
output_dir = "lewa/model/working/results"

# Save the model
CustomTrainer.save_model(output_dir)

# Save the tokenizer
CT_M3_Complete_tokenizer.save_pretrained(output_dir)

# Save training arguments
with open(f"{output_dir}/training_args.json", 'w') as f:
    json.dump(training_args.to_dict(), f)

# Save label mappings
with open(f"{output_dir}/label_mappings.json", 'w') as f:
    json.dump({"label2id": label2id, "id2label": id2label}, f)

print(f"Model and associated files saved to {output_dir}")

In [ ]:
# Load the model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("lewa/model/working/results")
tokenizer = AutoTokenizer.from_pretrained("lewa/model/working/results")

# Load label mappings
with open("lewa/model/working/results/label_mappings.json", 'r') as f:
    label_mappings = json.load(f)

id2label = label_mappings["id2label"]
print(id2label)

## SUBMISSION

In [ ]:
def merge_subwords_and_locations(tokens_and_labels):
    merged_words = []
    merged_labels = []
    current_word = []
    current_labels = []
    location_buffer = []

    for token, label in tokens_and_labels:
        if token.endswith('@@'):
            current_word.append(token[:-2])  # Remove '@@'
            current_labels.append(label)
        else:
            current_word.append(token)
            current_labels.append(label)

            # Merge subwords
            merged_word = ''.join(current_word)

            # Voting for the label
            if len(set(current_labels)) == 1:
                merged_label = current_labels[0]
            else:
                priority_order = ['B-LOC', 'I-LOC', 'E-LOC', 'S-LOC', 'O']
                merged_label = next(label for label in priority_order if label in current_labels)

            # Handle location merging
            if merged_label.endswith('-LOC'):
                if merged_label == 'B-LOC' or merged_label == 'S-LOC':
                    if location_buffer:
                        merged_words.append(' '.join(location_buffer))
                        merged_labels.append('B-LOC')
                        location_buffer = []
                    location_buffer.append(merged_word)
                elif merged_label == 'I-LOC' or merged_label == 'E-LOC':
                    location_buffer.append(merged_word)
                    if merged_label == 'E-LOC':
                        merged_words.append(' '.join(location_buffer))
                        merged_labels.append('B-LOC')
                        location_buffer = []
            else:
                if location_buffer:
                    merged_words.append(' '.join(location_buffer))
                    merged_labels.append('B-LOC')
                    location_buffer = []
                merged_words.append(merged_word)
                merged_labels.append(merged_label)

            # Reset for next word
            current_word = []
            current_labels = []

    # Handle any remaining location in the buffer
    if location_buffer:
        merged_words.append(' '.join(location_buffer))
        merged_labels.append('B-LOC')

    return list(zip(merged_words, merged_labels))

# # Usage
# merged_result = merge_subwords_and_locations(predicted_tokens)

# # Extract locations
# locations = [word for word, label in merged_result if label == 'B-LOC']
# print("\nExtracted locations:", locations)

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(**inputs).logits

    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

    tokens = []
    predicted_tokens = []

    locations = []
    current_location = []

    for token, prediction in zip(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]), predictions[0]):
        if int(prediction) == 0:  # Beginning of a new location
            current_location = [token]
        elif int(prediction) == 2:  # Inside a location
            if current_location:  # Make sure we started a location
                current_location.append(token)
        elif int(prediction) == 1:  # End of a location
            if current_location:  # Make sure we're inside a location
                current_location.append(token)
                locations.append(" ".join(current_location))
                current_location = []
        elif int(prediction) == 4:  # Single token location
            locations.append(token)
        else:
            current_location = []  # Reset if prediction is 'O' or anything else

        # Remove special tokens and clean up the text
        if token not in ['<s>', '</s>', '<unk>']:
            cleaned_token = token[1:] if token.startswith('Ġ') else token

            if token.startswith('##'):
                if predicted_tokens:
                    predicted_tokens[-1] = (predicted_tokens[-1][0] + cleaned_token, predicted_tokens[-1][1])
                continue

            tokens.append(cleaned_token)
            predicted_tokens.append((cleaned_token, id2label[str(prediction.item())]))

    # Usage
    merged_result = merge_subwords_and_locations(predicted_tokens)

    # Extract locations
    locations = [word for word, label in merged_result if label == 'B-LOC']

    # Extract unique locations and sort alphabetically
    unique_locations = sorted(set(locations))

    return unique_locations, tokens, predictions, predicted_tokens

In [ ]:
test = pd.read_csv("lewa/test.csv")

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.data.path.append('/usr/share/nltk_data/')

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '<URL>', text, flags=re.MULTILINE)

    # Remove user mentions
    text = re.sub(r'@\w+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z0-9\s\./\-_]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

#     # Lemmatize
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into a string
    processed_text = ' '.join(tokens)

    return processed_text

# Apply preprocessing to each text in your dataset
test['processed_text'] = test['text'].apply(preprocess_text)

In [ ]:
submission = []

for index, row in test.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index}")

    id = row['tweet_id']
    processed_text = row['processed_text']

    unique_locations, tokens, predictions, predicted_tokens = predict(processed_text)

    # Join locations with space, or use a single space if no locations
    locations_string = ' '.join(unique_locations) if unique_locations else ' '

    submission.append({'ID': id, 'Locations': locations_string})

# Create DataFrame from submission list
submission_df = pd.DataFrame(submission)

# Save to CSV
submission_df.to_csv('lewa/submission.csv', index=False)

In [ ]:
submission_df.head()